# Certifying ACLs and Firewall Rules

In [1]:
# Import packages and load questions
import pandas as pd
from IPython.display import display
from pandas.io.formats.style import Styler

from pybatfish.client.commands import *
from pybatfish.datamodel.answer import TableAnswer
from pybatfish.question import bfq, load_questions
from pybatfish.util import get_html
import logging

logging.getLogger('pybatfish').setLevel(logging.WARN)
load_questions()

In [2]:
# helper code to nicely print pandas frames
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)
# Prevent rendering text between '$' as MathJax expressions
pd.set_option('display.html.use_mathjax', False)

# UUID for CSS styles used by pandas styler.
# Keeps our notebook HTML deterministic when displaying dataframes
_STYLE_UUID = "pybfstyle"

class MyStyler(Styler):
    """A custom styler for displaying DataFrames in HTML"""

    def __repr__(self):
        return repr(self.data)

def show(df):
    """
    Displays a dataframe as HTML table.
    Replaces newlines and double-spaces in the input with HTML markup, and
    left-aligns the text.
    """
    if isinstance(df, TableAnswer):
        df = df.frame()

    # workaround for Pandas bug in Python 2.7 for empty frames
    if not isinstance(df, pd.DataFrame) or df.size == 0:
        display(df)
        return
    display(MyStyler(df).set_uuid(_STYLE_UUID).format(get_html)
            .set_properties(**{'text-align': 'left', 'vertical-align': 'top'}))

In [3]:
def make_acl(reference_flows, hostname, filter_name):
    """
    Turns the list of reference flows into an NXOS config file with the provided hostname and filter_name.
    """
    acl_lines = ["hostname {}".format(hostname), 
                 "ip access-list {}".format(filter_name)]
    seq_number = 10
    for flow in reference_flows:
        acl_lines.append("  {} {}".format(seq_number, flow))
        seq_number += 10
    acl_lines.append("  {} deny ip any any".format(seq_number))
    return "\n".join(acl_lines)

In [4]:
# initialize the snapshot we want to analyze
bf_session.init_snapshot("example/snapshot/", name="snapshot", overwrite=True)

# define the reference set flows that our ACL should allow
reference_flows = [
"permit ip 11.36.216.170/32 11.36.216.169/32",
"permit ip 11.36.216.176/32 11.36.216.179/32",
"permit ip 204.150.33.175/32 204.150.33.83/32",
"permit ip 205.248.59.64/32 205.248.59.67/32",
]

# turns the reference flows into a snapshot
# the hostname and filtername should correspond to the entities we want to evaluate
# we use Cisco-NXOS as the format for the reference ACL---the ACL int he snapshot can be in any format
nxos_acl = make_acl(reference_flows, "rtr-with-acl", "acl_in")
bf_session.init_snapshot_from_text(nxos_acl, platform="cisco-nx", snapshot_name="reference", overwrite=True)

'reference'

In [5]:
# Now, compare the two ACLs in the two snapshots
answer = bfq.compareFilters().answer(snapshot="snapshot", reference_snapshot="reference")
show(answer.frame())

,Node,Filter_Name,Line_Index,Line_Content,Line_Action,Reference_Line_Index,Reference_Line_Content
0,rtr-with-acl,acl_in,0,50 permit udp 117.186.185.0/24 range 49152 65535 117.186.185.0/24 eq 3784,PERMIT,4,50 deny ip any any
1,rtr-with-acl,acl_in,1,60 permit udp 117.186.185.0/24 range 49152 65535 117.186.185.0/24 eq 3785,PERMIT,4,50 deny ip any any
2,rtr-with-acl,acl_in,6,160 permit tcp 205.248.58.190/32 205.248.58.188/32 eq bgp,PERMIT,4,50 deny ip any any
3,rtr-with-acl,acl_in,7,180 permit tcp 205.248.57.174/32 205.248.57.173/32 eq bgp,PERMIT,4,50 deny ip any any
4,rtr-with-acl,acl_in,8,200 permit tcp 11.36.202.137/32 11.36.202.139/32 eq bgp,PERMIT,4,50 deny ip any any
5,rtr-with-acl,acl_in,9,220 permit tcp 11.36.202.142/32 11.36.202.141/32 eq bgp,PERMIT,4,50 deny ip any any
6,rtr-with-acl,acl_in,10,240 permit tcp 2.73.168.211/32 2.73.168.208/32 eq bgp,PERMIT,4,50 deny ip any any
7,rtr-with-acl,acl_in,11,260 permit tcp 212.58.171.208/32 212.58.171.208/32 eq bgp,PERMIT,4,50 deny ip any any
8,rtr-with-acl,acl_in,12,280 permit tcp 112.104.108.152/32 112.104.108.154/32 eq bgp,PERMIT,4,50 deny ip any any
9,rtr-with-acl,acl_in,13,300 permit tcp 218.8.104.14/32 218.8.104.15/32 eq bgp,PERMIT,4,50 deny ip any any
